In [1]:
import pandas as pd

df3 = pd.read_table('blast_uniprot_results.txt', header=None)
# the default outfmt 6 columns
default_outfmt6_cols = 'qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore'.strip().split(' ')
df3.columns = default_outfmt6_cols

In [3]:
len(df3)

1484868

In [20]:
# filter for evalue <= 0.001
df_filtered = df3[df3['evalue'] <= 10e-5]

# filter for evalue <= 0.001
df_filtered = df3[df3['pident'] >= 75]

# sort by bitscore
df_filtered.sort_values(by='bitscore', ascending=False, inplace=True)

/Users/stronglab2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
len(df_filtered)
#df_filtered.to_csv("Filtered_Uniprot_BLAST_Table.txt", sep='\t')

5022

In [22]:
df_filtered.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
710,YP_008318505.1,VFDB|VFG000671,99.177,486,4,0,1,486,1,486,0.0,961.0
636630,YP_004286239.1,VFDB|VFG000109,100.000,399,0,0,1,399,1,399,0.0,831.0
1011892,YP_001449320.1,VFDB|VFG002121,85.033,461,47,3,1,459,1,441,0.0,758.0
1028668,NP_795593.1,VFDB|VFG000965,99.703,337,1,0,1,337,1,337,0.0,683.0
420709,NP_438157.1,VFDB|VFG000965,99.703,337,1,0,1,337,1,337,0.0,683.0


In [23]:
hits = df_filtered['qseqid']
uniq_hits = list(set(hits))
len(uniq_hits)

3993

### Load in Uniprot Protein Sequence IDs

In [9]:
from Bio import SeqIO
ids = []
names = []
for seq_record in SeqIO.parse("viral.1.protein.faa", "fasta"):
    ids.append(seq_record.id)
    names.append(seq_record.description)
    
viral_names = [x.split('[', 1)[1].split(']')[0] for x in names]

In [10]:
df = pd.DataFrame({'IDs': ids, 
                   'Viral Names': viral_names, 
                   'CompleteRecord': names}, columns=['IDs','Viral Names', 'Complete Record'])

In [11]:
y = df['Viral Names'].str.contains('phage')
df2=df[y]
len(df2)

155606

In [12]:
df2['Virulence'] = [1 if x in uniq_hits else 0 for x in df2['IDs']]

/Users/stronglab2/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
## Group by
test = df2.groupby('Viral Names').sum()

In [14]:
## Group by Bacterial Genus
genus_name = test.index
demo2 = [x.split(' ',1)[0] for x in genus_name]

In [15]:
test['Genus'] = demo2

In [16]:
Virulence_Avg = test.groupby('Genus').mean()
Virulence_Avg = Virulence_Avg.sort_values(by=['Virulence'], ascending=False)
Virulence_Avg.columns = ['Avg Number of Virulent Matches']
Virulence_Avg.head()

,Avg Number of Virulent Matches
Genus,
Halorubrum,8.000000
Nitrincola,7.000000
Prochlorococcus,6.333333
Sinorhizobium,5.800000
Aeromonas,4.000000


In [17]:
Number_of_phages = test.groupby('Genus').count()
Number_of_phages = Number_of_phages.sort_values(by=['Virulence'], ascending=False)
Number_of_phages.columns = ['Number of Phages in Genus']
Number_of_phages.head()

,Number of Phages in Genus
Genus,
Mycobacterium,249
Pseudomonas,112
Escherichia,92
Bacillus,82
Salmonella,67


In [18]:
### Number of Viruses over 30 in Genus
x = Number_of_phages[Number_of_phages['Number of Phages in Genus']>=30]
abundant_phages = x.index

In [19]:
Abundant_df = Virulence_Avg[(Virulence_Avg.index).isin(abundant_phages)]
Abundant_df

,Avg Number of Virulent Matches
Genus,
Bacillus,2.987805
Mycobacterium,2.887550
Escherichia,2.760870
Salmonella,2.223881
Vibrio,1.939394
Enterobacteria,1.881356
Pseudomonas,1.866071
Gordonia,1.861538
Lactococcus,1.685714
